## Define default variables
Overwritten by widget values if the notebook is viewed in Mercury

In [98]:
exhibition_data_type = "all"
organisation = "The Museum of Modern Art"
selected_year = "1979"


print(f"exhibition_data_type={exhibition_data_type}")
print(f"organisation={organisation}")
print(f"selected_year={selected_year}")

exhibition_data_type=all
organisation=The Museum of Modern Art
selected_year=1979


# Linked Art - Data Shape - Exhibitions
This notebook shows the *shape* of data for exhibitions included in the selected Linked Art dataset.

## Purpose
Understanding the shape of data helps to identify features of the data that can be presented in a useable, browsable sectioned display.


It is proposed that a characteristic of the data that results in an even distribution of records, will correspond to a useable, browsable breakdown of the data that can be presented in a sectioned display.

## How-to

## Visual Studio Code - Jupyter plugin
https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter 

If you have the Jupyter notebook extension, you can run the notebooks in Visual Studio Code editor:
- open and run the `variables.ipynb` notebook
- open and run the `persons.ipynb` notebook

## Jupyter Notebook
https://jupyter.org/ 


The run the Jupyter notebook in the browser, use the following command:
- `jupyter notebook`
- open and run the `variables.ipynb` notebook
- open and run the `exhibitions.ipynb` notebook

### Mercury Web application
https://mljar.com/mercury/ 

The run this notebook as a web app using Mercury, use the following commands in the root directory:

- `mercury run variables.ipynb`
- `mercury run exhibitions.ipynb`

If you want to modify the notebook and view changes dynamically in the Mercury app, use the `watch` commands in the root directory:
- `mercury watch variables.ipynb`
- `mercury watch exhibitions.ipynb`


Go the to URL provided by Mercury, e.g. http://127.0.0.1:8000/ 
- Run the Notebook variables notebook
- Open the Persons notebook, and click the green `Run` button in the left-hand column

In [99]:
import json
import os
from sortedcontainers import SortedDict
import plotly.express as px

%store -r summary_data_dir_activity
%store -r activity_all_file

%store -r events_nonmoma
%store -r json_suffix
%store -r linked_data_filepath_group
%store -r datavis_venue_exhibitions 

%store -r summary_data_dir_person
%store -r persons_all_file

In [100]:
def create_histogram(data, x_label, y_label, histogram_title, entity_type, selected_field, ignore_list=[], histogram_type="", sort_type=0, linked_data_filepath_group=""): 

    number_dict = {}
    # iterate through selected entity type
    for entity in data.get(entity_type):

        if selected_field in entity:
            property = entity.get(selected_field)


            if property in ignore_list:
                continue

            if histogram_type == "org_year":
                property = property.split("-")[0]

            if histogram_type == "org_year_month":
                property = property.split("-")[1]
            # 
            if property in number_dict:
                number_dict[property] += 1
            else:
                number_dict[property] = 1

    if histogram_type == "org":
        # if organisation histogram - get organisation name and create dict using name
        number_dict_updated = {}
        for property in number_dict:
            filename = property.split("/").pop() + ".json"
            with open(os.path.join(linked_data_filepath_group, filename), 'r') as group_file:
                groupdata = json.load(group_file)
                label = groupdata.get("_label")
                number_dict_updated[label] = number_dict[property]
        number_dict = number_dict_updated



    # sort the dictionary by frequency or label - determined by var sort_type
    number_dict = dict(sorted(number_dict.items(), key=lambda item: item[sort_type]))

    fig = px.bar(x=number_dict.keys(), y=number_dict.values(), labels={"x": x_label, "y": y_label},title=histogram_title)
    return fig

In [101]:
## read data file
file_dir = summary_data_dir_activity
file_entity = activity_all_file

data = {}

   
file_type = "all"

if exhibition_data_type == "MoMA":
    file_type = "moma"
elif exhibition_data_type == "non-MoMA":
    file_type = "nonmoma"

print("Dataset selected: " + file_type)

data_file = "events_" + file_type + ".json"

with open(os.path.join(summary_data_dir_activity, data_file), 'r') as json_file: 
    data = json.load(json_file)


Dataset selected: all




## Number of exhibitions by organisation

In [102]:

x_label = "Organisation"
y_label = "Number of exhibitions"
histogram_title = "Number of exhibitions by organisation - " + organisation

entity_type = "events"
selected_field = "org"
ignore_list = [""]

histogram = create_histogram(data, x_label, y_label, histogram_title, "events", selected_field, ignore_list,"org", 1,linked_data_filepath_group)
histogram.show()



## Exhibitions by year for selected organisation

In [103]:
data_selected_org = []


ex_by_org = {}
ex_by_org_updated = {}

for event in data.get("events"):
    org = event.get("org")
    if org in ex_by_org: 
        ex_by_org[org].append(event)
    else:
        ex_by_org[org] = []
        ex_by_org[org].append(event)

    

for org_id in ex_by_org:
    filename = org_id.split("/").pop() + ".json"
    with open(os.path.join(linked_data_filepath_group, filename), 'r') as group_file:
        groupdata = json.load(group_file)
        label = groupdata.get("_label")
        ex_by_org_updated[label] = ex_by_org[org_id]
ex_by_org = ex_by_org_updated

        
# get data for selected org and calculate number distribution
if organisation in ex_by_org.keys():
    data_selected_org = ex_by_org[organisation]


histogram_title = "Exhibitions by year for organisation: " + organisation

histogram = create_histogram({"events":data_selected_org}, x_label, y_label, histogram_title, entity_type, "start", ignore_list,"org_year", 0,linked_data_filepath_group)
histogram.show()



## Exhibitions for selected year by month
Histogram showing breakdown of exhibitions for a selected year, by month.

In [105]:

data_selected_year = []

for event in data_selected_org:
    start_year = event.get("start").split("-")[0]

    if int(start_year) == int(selected_year):
        data_selected_year.append(event)


if len(data_selected_year) > 0:
    x_label = "Month"
    histogram_title = f"Exhibitions by month :: Organisation: {organisation} :: Year: {selected_year}"

 
    histogram = create_histogram({"events":data_selected_year}, x_label, y_label, histogram_title, entity_type, "start", ignore_list,"org_year_month", 0,linked_data_filepath_group)
    histogram.show()
